In [1]:
from rdflib import Graph
g = Graph()
g.parse("../../resources/eurovoc-skos-ap-eu.rdf")

<Graph identifier=N7625426e505d4a3087b554750e682fd3 (<class 'rdflib.graph.Graph'>)>

In [2]:
g

<Graph identifier=N7625426e505d4a3087b554750e682fd3 (<class 'rdflib.graph.Graph'>)>

In [5]:
for i, (s, p, o) in enumerate(g):
    print((s, p, o))
    if i == 10:
        break

(rdflib.term.URIRef('http://eurovoc.europa.eu/81'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#altLabel'), rdflib.term.Literal("women's position", lang='en'))
(rdflib.term.URIRef('http://eurovoc.europa.eu/5176'), rdflib.term.URIRef('http://www.w3.org/2008/05/skos-xl#prefLabel'), rdflib.term.URIRef('http://eurovoc.europa.eu/110652'))
(rdflib.term.URIRef('http://eurovoc.europa.eu/438643'), rdflib.term.URIRef('http://purl.org/dc/terms/type'), rdflib.term.URIRef('http://publications.europa.eu/resource/authority/label-type/STANDARDLABEL'))
(rdflib.term.URIRef('http://eurovoc.europa.eu/3035'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#altLabel'), rdflib.term.Literal('vēlēšanu likumības pārbaude', lang='lv'))
(rdflib.term.URIRef('http://eurovoc.europa.eu/156298'), rdflib.term.URIRef('http://www.w3.org/2008/05/skos-xl#literalForm'), rdflib.term.Literal('association', lang='da'))
(rdflib.term.URIRef('http://eurovoc.europa.eu/496773'), rdflib.term.URIRef('http://www.w3

In [23]:
from rdflib import Graph, Literal, URIRef
from unidecode import unidecode
import logging


# Function to search concepts by literal string
def search_concepts_fast(graph, literal_string):
    results = []
    # Search for the literal in the graph
    for s, p, o in graph.triples((None, URIRef('http://www.w3.org/2004/02/skos/core#prefLabel'), Literal(literal_string, lang='nl'))):
        results.append(s)
    return results

def search_concepts_slow(graph, literal_string):
    results = []
    # Iterate over all triples with the prefLabel property
    for s, p, o in graph.triples((None, URIRef('http://www.w3.org/2004/02/skos/core#prefLabel'), None)):
        # Check if the object's value matches the literal_string, ignoring case
        if o.language == 'nl' and unidecode(o.lower()) == unidecode(literal_string.lower()):
            results.append(s)
    
    if results:
        return results
    
    results = []
    # Iterate over all triples with the prefLabel property
    for s, p, o in graph.triples((None, URIRef('http://www.w3.org/2004/02/skos/core#altLabel'), None)):
        # Check if the object's value matches the literal_string, ignoring case
        if o.language == 'nl' and unidecode(o.lower()) == unidecode(literal_string.lower()):
            results.append(s)
    
    if results:
        return results
    
    results = []
    # Iterate over all triples with the prefLabel property
    for s, p, o in graph.triples((None, URIRef('http://www.w3.org/2004/02/skos/core#prefLabel'), None)):
        # Check if the object's value matches the literal_string, ignoring case
        if o.language == 'nl' and "".join(unidecode(literal_string.lower()).split()) in "".join(unidecode(o.lower()).split()):
            results.append(s)
    
    if results:
        return results
    
    results = []
    # Iterate over all triples with the prefLabel property
    for s, p, o in graph.triples((None, URIRef('http://www.w3.org/2004/02/skos/core#altLabel'), None)):
        # Check if the object's value matches the literal_string, ignoring case
        if o.language == 'nl' and "".join(unidecode(literal_string.lower()).split()) in "".join(unidecode(o.lower()).split()):
            results.append(s)
    
    if results:
        return results
    
    return []
    


# Function to retrieve broader concepts
def find_broader_concepts(graph, concept_uri):
    broader_concepts = []
    for s, p, o in graph.triples((concept_uri, URIRef('http://www.w3.org/2004/02/skos/core#broader'), None)):
        broader_concepts.append((s, p, o))
        broader_concepts.extend(find_broader_concepts(graph, o))  # Recursively find broader concepts
    return broader_concepts

# Function to find Top Concept Of the broadest concept
def find_top_concept_of(graph, concept_uri):
    top_concept_of = []
    for s, p, o in graph.triples((concept_uri, URIRef('http://www.w3.org/2004/02/skos/core#topConceptOf'), None)):
        top_concept_of.append((s, p, o))
        top_concept_of.extend(find_top_concept_of(graph, o))  # Recursively find broader concepts
    return top_concept_of

# Function to find Domain of a Top Concept
def find_domain(graph, top_concept_uri):
    domain = []
    for s, p, o in graph.triples((top_concept_uri, URIRef('http://publications.europa.eu/ontology/euvoc#domain'), None)):
        domain.append((s, p, o))
        domain.extend(find_domain(graph, o))  # Recursively find broader concepts
    return domain

# Function to search concepts by literal string
def search_literal(graph, domain_uri):
    results = []
    # Search for the prefLabel property in the given domain_uri specifically in Dutch
    for s, p, o in graph.triples((domain_uri, SKOS.prefLabel, None)):
        if o.language == 'nl':
            results.append((s, p, str(o)))  # Convert Literal to string for better readability
    return results


def get_domain(query):
    # Example usage
    concept_search = query.lower()
    concepts = search_concepts_fast(g, concept_search)
    if not concepts:
        concepts = search_concepts_slow(g, concept_search)
    # print("Found concepts:", concepts)
    # print(search_literal(g, concepts[0])[0][2])
    # print("\n")

    # Concepts error handling
    if len(concepts) == 0:
        raise ValueError("Concept not found")
    elif len(concepts) > 1:
        logging.warning(f"More than 1 concept found? {[search_literal(g, c)[0][2] for c in concepts]}\nPicking first one.")

    concept = concepts[0]

    # Get broader concepts for a found concept
    broader = find_broader_concepts(g, concept)
    # print("Broader concepts:", broader)

    if broader:
        # print(search_literal(g, broader[-1][2])[0][2])
        concept = broader[-1][2]

    # print("\n")

    # Get the top concept of the broadest concept
    top = find_top_concept_of(g, concept)
    # print("Top concepts:", top)
    # print(search_literal(g, top[-1][2])[0][2])

    # print("\n")

    # Get the top concept of the broadest concept
    domain = find_domain(g, top[-1][2])
    # print("Domain:", domain)
    # print(search_literal(g, domain[0][2])[0][2])

    return search_literal(g, domain[0][2])[0][2]

get_domain("COMMUNAUTAIRE HULP")

'16 ECONOMIE'

In [24]:
import json
import os

folder_path = "../../src/parlementaire_stukken/json"

kamerstukken = []
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        kamerstukken.append((filename, data))
print(len(kamerstukken))

4015


In [34]:
import json

from common.text_corrections import fix_name

with open("../../members.json", "r") as fp:
    name_metadata = json.load(fp)
name_to_party = {fix_name(n["name"], swap_first_last_name=True): n["party"] for n in name_metadata}

In [37]:
# Pseudocode
from collections import defaultdict
from tqdm import tqdm
from common.text_corrections import fix_name


parties_to_domains = defaultdict(lambda: defaultdict(int))
for stukname, kamerstuk in tqdm(kamerstukken):
    try:
        hoofddescriptor = kamerstuk["Descriptoren, trefwoorden"]["Eurovoc-hoofddescriptor"]["NL"]
        auteurs = kamerstuk["Kamer en/of Senaat"]["Hoofddocument"][-1]["Auteur"]

        try:
            domain = get_domain(hoofddescriptor)
        except ValueError:
            domain = hoofddescriptor
        parties_seen = set()
        for auteur in auteurs:
            if auteur["Partij"] == "ZZZ":
                parties_to_domains["ZZZ"][domain] += 1
            else:
                name = fix_name(f"{auteur['Naam']} {auteur['Voornaam']}", swap_first_last_name=False)
                if name in name_to_party:
                    party = name_to_party[name]
                else:
                    party = auteur["Partij"]
                if party not in parties_seen:
                    parties_to_domains[party][domain] += 1
                    parties_seen.add(party)
    except KeyError:
        print(stukname)


  0%|          | 0/4015 [00:00<?, ?it/s]

  0%|          | 3/4015 [00:00<14:02,  4.76it/s]

55K0374.json


  1%|          | 27/4015 [00:05<16:04,  4.14it/s]

55K1899.json


...

 97%|█████████▋| 3897/4015 [08:07<00:29,  3.96it/s]

55K2949.json


 97%|█████████▋| 3907/4015 [08:09<00:25,  4.21it/s]

55K2408.json


100%|██████████| 4015/4015 [08:17<00:00,  8.07it/s]


In [38]:
parties_to_domains

defaultdict(<function __main__.<lambda>()>,
            {'ZZZ': defaultdict(int,
                         {'08 INTERNATIONALE BETREKKINGEN': 98,
                          '66 ENERGIE': 55,
                          '28 SOCIALE VRAAGSTUKKEN': 158,
                          '32 OPVOEDING, ONDERWIJS EN COMMUNICATIE': 31,
                          '04 POLITIEK': 87,
                          '12 RECHT': 121,
                          '24 FINANCIËN': 146,
                          '44 WERKGELEGENHEID EN ARBEID': 49,
                          '48 TRANSPORT': 21,
                          '52 MILIEU': 22,
                          '40 ONDERNEMING EN CONCURRENTIE': 11,
                          'GEESTELIJK GEHANDICAPTE': 1,
                          '72 GEOGRAFIE': 3,
                          '16 ECONOMIE': 19,
                          '20 ECONOMIE, VERKEER EN HANDELSVERKEER': 20,
                          '68 INDUSTRIE': 8,
                          'PROGRAMMAWET': 5,
                      

In [42]:
import plotly.graph_objects as go

# Setup data for absolute amounts
def plot_absolute(data):
    fig = go.Figure()
    for party, subjects in data.items():
        fig.add_trace(go.Bar(
            name=party,
            x=list(subjects.keys()),
            y=list(subjects.values()),
        ))
    fig.update_layout(
        title="Absolute Amounts per Subject by Party",
        xaxis_title="Subject",
        yaxis_title="Amount",
        barmode='group',
        xaxis={'categoryorder':'total descending'},
        height=800
    )
    fig.show()

plot_absolute(parties_to_domains)

In [48]:
colors = {
    "DéFI": "#FFA500",       # Orange
    "N-VA": "#FFD700",       # Yellow
    "Les Engagés": "#FF8C00",# Darker Orange
    "VB": "#000000",         # Black
    "PVDA": "#FF0000",       # Bright Red
    "Open Vld": "#1E90FF",   # Dodger Blue
    "PTB": "#CD5C5C",        # Indian Red
    "MR": "#87CEEB",         # Sky Blue
    "Vooruit": "#DC143C",    # Crimson
    "Ecolo": "#228B22",      # Forest Green
    "Groen": "#32CD32",      # Lime Green
    "CD&V": "#FF6347",       # Tomato
    "PS": "#B22222",         # Firebrick Red
    "cd&v": "#FFF",          # White
    "ZZZ": "#1D1D1D"         # Grey
}


def calculate_relative(data):
    relative_data = {}
    for party, subjects in data.items():
        total = sum(subjects.values())
        relative_data[party] = {subject: value / total for subject, value in subjects.items()}
    return relative_data

def plot_relative(data):
    relative_data = calculate_relative(data)
    fig = go.Figure()
    for party, subjects in relative_data.items():
        fig.add_trace(go.Bar(
            name=party,
            x=list(subjects.keys()),
            y=list(subjects.values()),
            marker_color=colors[party]
        ))
    fig.update_layout(
        title="Relative Amounts per Subject by Party",
        xaxis_title="Subject",
        yaxis_title="Percentage",
        barmode='group',
        xaxis={'categoryorder':'total descending'},
        height=1080,
        width=1920
    )
    fig.show()

plot_relative(parties_to_domains)